# Classifying Color Images - Dogs, Cats
### Author: Samuel Adamson
### Tensorflow, Google ML Datasets, Numpy, MatPlotLib
### Last Edited 01/01/2022

In [ ]:
# Imports
import tensorflow as tf
import tensorflow.keras.preprocessing.image as imageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os

# Set logging for errors only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'# 

Download Data from Google ML Dataset <br>
Unzip Data <br>
Store directories with different data types: <br>
&emsp; Training Data <br>
&emsp; Validation Data <br>
<br>
Evaluate data - Number of data points in each category

In [ ]:
# Data url for download
DATA_URL = 'https://storage/googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
# Download, store in directory, extract from .zip
dir = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=DATA_URL, extract=True)

# Store path to each type of data (training/validation)
data_dir = os.path.join(os.path.dirname(dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(os.path.dirname(data_dir), 'train')
valid_dir = os.path.join(os.path.dirname(data_dir), 'validation')

# Store path to each type of image classification
train_cats_dir = os.path.join(os.path.dirname(train_dir), 'cats')
train_dogs_dir = os.path.join(os.path.dirname(train_dir), 'dogs')
valid_cats_dir = os.path.join(os.path.dirname(valid_dir), 'cats')
valid_dogs_dir = os.path.join(os.path.dirname(valid_dir), 'dogs')


# Number of validation images
num_val = len(os.listdir(valid_cats_dir)) + len(os.listdir(valid_dogs_dir))
print(f'Number of validation images: {num_val}')

# Number of training images
num_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
print(f'Number of training images: {num_train}')
print('---')

# Number of training / validation cats
num_train_cats = len(os.listdir(train_cats_dir))
num_valid_cats = len(os.listdir(valid_cats_dir))
tot_cats = num_train_cats + num_valid_cats
print(f'Number of training cat images: {num_train_cats}')
print(f'Number of validation cat images: {num_valid_cats}')
print(f'Total number of cat images: {tot_cats}')
print('---')

# Number of training / validation dogs
num_train_dogs = len(os.listdir(train_dogs_dir))
num_valid_dogs = len(os.listdir(valid_dogs_dir))
tot_dogs = num_train_dogs + num_valid_dogs
print(f'Number of training dog images: {num_train_dogs}')
print(f'Number of validation dog images: {num_valid_dogs}')
print(f'Total number of dog images: {tot_dogs}')